In [23]:
import pandas as pd

import sys
sys.path.append("/home/paolo/airflow/dags")

from custom_code import custom_lib as cl
import re

fp = cl.path_utils.get_today_remotive()
print(fp)

df = pd.read_json(fp)
df.head()
print(df.category.unique())

sw_df = df[df["category"] == "Software Development"]
sw_df.head()
sw_df.candidate_required_location.unique()
sw_df["location"] = sw_df.candidate_required_location.str.lower()

weg = sw_df[
    (sw_df["location"].str.contains("europe"))
    | (sw_df["location"].str.contains("germany"))
    | (sw_df["location"].str.contains("worldwide"))
]

print(weg.salary.unique())
currency_api = cl.currency_api.CachedCurrencyAPI()
t = currency_api.convert_usd_to_euros(50.000)

def salary_cleaner(s: str):
    out = s.lower()
    number_k = out.count("k")

    is_hourly = False
    is_monthly = False
    is_dollar = False
    is_euro = False

    if number_k >= 1:
        out = out.replace("k", "000")

    if "hour" in out:
        is_hourly = True
    if "month" in out:
        is_monthly = True

    if "$" in out or "dollar" in out:
        is_dollar = True

    if "€" in out or "euro" in out:
        is_euro = True

    if is_hourly and is_monthly:
        raise Exception(f"Is it hourly on monthly salary?? {s}") 

    if is_dollar and is_euro:
        raise Exception(f"Is it in dollar or euro? {s}")

    numbers = re.findall("\d+", out)
    salary = -1
    if len(numbers) == 0:
        return float(salary)
    elif len(numbers) == 1:
        salary = float(numbers[0])
    # If it's a range, take average
    elif len(numbers) == 2:
        salary = (float(numbers[0]) + float(numbers[1])) / 2
    else:
        raise Exception(f"Unhandled case, 3 or more salary numbers? {s}")
    if is_dollar:
        salary_in_euros = currency_api.convert_usd_to_euros(salary)
    # Else assume it's euros
    else:
        salary_in_euros = salary
    
    if is_monthly:
        annual_salary_in_euros = salary_in_euros * 12
    elif is_hourly:
        # 8 hours / day, 20 days / month, 12 months / year
        annual_salary_in_euros = salary_in_euros * 8 * 20 * 12
    else:
        annual_salary_in_euros = salary_in_euros
    return annual_salary_in_euros

weg["salary"] = weg["salary"].apply(salary_cleaner)


print(weg.salary.unique())

weg.head()

/data/remotive/2023-01-23_remotive.json
['Design' 'Software Development' 'All others' 'Human Resources'
 'Finance / Legal' 'DevOps / Sysadmin' 'Product' 'Sales' 'Marketing'
 'Customer Service' 'Data' 'QA' 'Business' 'Writing']
['' '€63k' '$50k-$70k' '$125k - $200k' '$60k-$108k' '$50K - $60K'
 '$80k-$180k' '$50k-$70K' '$60k - $80k' '$40 - $80 /hour' '$30 - $50k'
 '$48k-$96k' '€5495 /month' '$110-190 hourly' '$65k-$80k' '€61k'
 '€6k/month']
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
[-1.00000000e+00  6.30000000e+04  5.51910000e+04  1.49475625e+05
  7.72674000e+04

/tmp/ipykernel_36498/1136386762.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sw_df["location"] = sw_df.candidate_required_location.str.lower()
/tmp/ipykernel_36498/1136386762.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weg["salary"] = weg["salary"].apply(salary_cleaner)


,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,company_logo_url,location
33,1565263,https://remotive.com/remote-jobs/software-dev/...,Front-end Developer ( React/Angular ),Keeper Solutions,https://remotive.com/job/1565263/logo,Software Development,"[big data, CSS, developer, html, javascript, j...",full_time,2023-01-21T13:39:53,Europe,-1.000,<p><br>Keeper Solutions is a market-leading so...,NaN,europe
70,1557608,https://remotive.com/remote-jobs/software-dev/...,Senior Full-Stack Engineer,Passionfroot,https://remotive.com/job/1557608/logo,Software Development,"[devops, javascript, postgresql, react, growth...",full_time,2023-01-20T11:40:08,Germany,-1.000,"<div class=""h3"">🔴 About Passionfroot</div>\n<p...",NaN,germany
133,1501053,https://remotive.com/remote-jobs/software-dev/...,Frontend Developer (JavaScript + React),Toggl Hire,https://remotive.com/job/1501053/logo,Software Development,"[AWS, backend, developer, frontend, golang, gr...",full_time,2023-01-18T10:30:38,"Europe, European timezones",63000.000,<p>We are looking for a Frontend Developer to ...,https://remotive.com/job/1501053/logo,"europe, european timezones"
137,1359476,https://remotive.com/remote-jobs/software-dev/...,Senior Full Stack Developer,Proxify,https://remotive.com/job/1359476/logo,Software Development,"[backend, developer, frontend, fullstack, mong...",contract,2023-01-18T07:45:49,Worldwide,55191.000,"<p>About us: </p>\n<p> </p>\n<p>""Talent has no...",https://remotive.com/job/1359476/logo,worldwide
141,1350221,https://remotive.com/remote-jobs/software-dev/...,"Senior Software Engineer, Python",AssemblyAI,https://remotive.com/job/1350221/logo,Software Development,"[api, AWS, backend, C, C#, C++, cloud, develop...",full_time,2023-01-18T05:56:25,"North America, Americas, LATAM, Europe, USA, U...",149475.625,"<p class=""h2""><strong>The Role</strong></p>\n<...",https://remotive.com/job/1350221/logo,"north america, americas, latam, europe, usa, u..."
